In [1]:
import sagemaker
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator

sess = sagemaker.Session()
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)

from sagemaker.image_uris import retrieve
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Parámetros de configuración
bucket = 'bakery-project'
prefix = 'data'
role = sagemaker.get_execution_role()
region = sess.boto_region_name
endpoint_name_prefix = "xgb-smart-bakery"

train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
predictor_instance_count = 1
predictor_instance_type = "ml.m4.xlarge"
clarify_instance_count = 1
clarify_instance_type = "ml.m4.xlarge"

In [3]:
# URIs de los datos de entrenamiento, validación y test en S3
train_data_uri = f's3://{bucket}/{prefix}/train.csv'
validation_data_uri = f's3://{bucket}/{prefix}/validation.csv'
test_data_uri = f's3://{bucket}/{prefix}/test.csv'

In [4]:
# Configura el container de XGBoost proporcionado por SageMaker
container = sagemaker.image_uris.retrieve('xgboost', sagemaker.Session().boto_region_name, version='1.2-1')

In [5]:
# Configura el estimador de SageMaker
xgb_estimator = Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    max_run=3600,
    output_path=f's3://{bucket}/{prefix}/output'
)

In [6]:
# Define los hiperparámetros base
xgb_estimator.set_hyperparameters(
    objective='reg:squarederror',
    num_round=100
)

In [7]:
# Define los rangos de búsqueda de hiperparámetros
hyperparameter_ranges = {
    'max_depth': IntegerParameter(3, 10),
    'eta': ContinuousParameter(0.01, 0.2),
    'subsample': ContinuousParameter(0.5, 1.0),
    'colsample_bytree': ContinuousParameter(0.5, 1.0)
}

# Define los datos de entrenamiento
train_input = TrainingInput(s3_data=train_data_uri, content_type='csv')
validation_input = TrainingInput(s3_data=validation_data_uri, content_type='csv')


In [8]:
# Configura el tunning de hiperparámetros
tuner = HyperparameterTuner(
    estimator=xgb_estimator,
    objective_metric_name='validation:rmse',  # Métrica objetivo
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=20,  # Número máximo de trabajos de entrenamiento
    max_parallel_jobs=3,  # Número máximo de trabajos paralelos
    objective_type='Minimize',  # Tipo de objetivo (minimizar RMSE)
    base_tuning_job_name='xgb-tuning-job'  # Nombre base para el trabajo de tuning
)

# Inicia el trabajo de tuning
tuner.fit({'train': train_input, 'validation': validation_input})
tuner.wait()

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


...........................................................................................!
!


In [9]:
# Resumen de resultados del tuning
df_tuner = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.job_name).dataframe()
df_tuner = df_tuner[df_tuner["FinalObjectiveValue"]>-float('inf')].sort_values("FinalObjectiveValue", ascending=False)
df_tuner

,colsample_bytree,eta,max_depth,subsample,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
14,0.954847,0.010000,4.0,0.689161,xgb-tuning-job-240521-0000-006-f9c609de,Completed,20.039049,2024-05-21 00:03:48+00:00,2024-05-21 00:04:21+00:00,33.0
1,1.000000,0.098496,3.0,0.500000,xgb-tuning-job-240521-0000-019-63ff2fb1,Completed,15.139470,2024-05-21 00:07:41+00:00,2024-05-21 00:08:10+00:00,29.0
0,1.000000,0.107253,3.0,1.000000,xgb-tuning-job-240521-0000-020-26955b10,Completed,15.020490,2024-05-21 00:07:48+00:00,2024-05-21 00:08:17+00:00,29.0
2,1.000000,0.200000,10.0,0.566182,xgb-tuning-job-240521-0000-018-fdcd44df,Completed,14.939930,2024-05-21 00:07:08+00:00,2024-05-21 00:07:42+00:00,34.0
16,0.571553,0.194038,8.0,0.741706,xgb-tuning-job-240521-0000-004-0b0d8c10,Completed,14.849100,2024-05-21 00:03:44+00:00,2024-05-21 00:04:13+00:00,29.0
9,0.698490,0.060672,4.0,0.989113,xgb-tuning-job-240521-0000-011-58a70795,Completed,14.691420,2024-05-21 00:05:24+00:00,2024-05-21 00:05:55+00:00,31.0
7,0.534006,0.098406,9.0,0.728753,xgb-tuning-job-240521-0000-013-0fb6622f,Completed,14.561250,2024-05-21 00:06:01+00:00,2024-05-21 00:06:35+00:00,34.0
3,1.000000,0.093721,10.0,0.627021,xgb-tuning-job-240521-0000-017-8a60a5c2,Completed,14.520060,2024-05-21 00:06:59+00:00,2024-05-21 00:07:33+00:00,34.0
17,0.898613,0.049503,4.0,0.692777,xgb-tuning-job-240521-0000-003-32fdf449,Completed,14.371130,2024-05-21 00:01:37+00:00,2024-05-21 00:03:31+00:00,114.0
11,0.959801,0.063878,4.0,0.741019,xgb-tuning-job-240521-0000-009-37e59970,Completed,14.310680,2024-05-21 00:04:38+00:00,2024-05-21 00:05:13+00:00,35.0


In [10]:
# Obtiene el mejor modelo del tuner
best_estimator = tuner.best_estimator()

predictor = best_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    serializer=CSVSerializer(),  # Serializer para enviar datos en formato CSV
    deserializer=CSVDeserializer()  # Deserializer para recibir la respuesta en formato CSV
)


2024-05-21 00:06:58 Starting - Found matching resource for reuse
2024-05-21 00:06:58 Downloading - Downloading the training image
2024-05-21 00:06:58 Training - Training image download completed. Training in progress.
2024-05-21 00:06:58 Uploading - Uploading generated training model
2024-05-21 00:06:58 Completed - Resource reused by training job: xgb-tuning-job-240521-0000-017-8a60a5c2
-----!

In [11]:
import numpy as np
import pandas as pd

# Datos para predicción
new_data_uri = f's3://{bucket}/{prefix}/Oct_s1.csv'
new_data = pd.read_csv(new_data_uri)

new_data = new_data.drop(columns=['Quantity'])

# Convertir el dataframe a un formato compatible (CSV en un string)
payload = new_data.to_csv(index=False, header=False)

# Realizar la predicción
predictions = predictor.predict(payload)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [12]:
# Aplanar la lista de predicciones
flat_predictions = [item for sublist in predictions for item in sublist]

# Convertir las predicciones a DataFrame con una sola columna
predictions_df = pd.DataFrame({'Prediction': flat_predictions})

# Mostrar el DataFrame resultante
print(predictions_df)

               Prediction
0       9.410957336425781
1       5.070960521697998
2       43.99121856689453
3     -0.8644360303878784
4    -0.35417336225509644
..                    ...
842      2.91869854927063
843    3.1801259517669678
844    13.755523681640625
845     21.38402557373047
846     5.482120990753174

[847 rows x 1 columns]


In [32]:
# Añadir la columna de predicciones al DataFrame de test_data
test_data_with_predictions = pd.concat([test_data, predictions_df], axis=1)
test_data_with_predictions['Prediction'] = test_data_with_predictions['Prediction'].astype(float)
test_data_with_predictions['Prediction'] = np.floor(test_data_with_predictions['Prediction']).astype(int)

In [33]:
test_data_with_predictions['Date'] = test_data_with_predictions['Month'].astype(int)*100+test_data_with_predictions['Day'].astype(int)
test_data_with_predictions['Date'] = pd.to_datetime(test_data_with_predictions['Date'], format='%Y%m%d')
test_data_with_predictions['Date'] = test_data_with_predictions['Date'].dt.strftime('%d/%m/%y')

test_data_with_predictions

,Year,Month,Day,Avg_temp,Menu_Code,Prediction,Date
0,2022,202210,1,63.4,146,9,01/10/22
1,2022,202210,1,63.4,82,5,01/10/22
2,2022,202210,1,63.4,2,43,01/10/22
3,2022,202210,1,63.4,116,-1,01/10/22
4,2022,202210,1,63.4,139,-1,01/10/22
...,...,...,...,...,...,...,...
842,2022,202210,7,55.1,105,2,07/10/22
843,2022,202210,7,55.1,16,3,07/10/22
844,2022,202210,7,55.1,135,13,07/10/22
845,2022,202210,7,55.1,8,21,07/10/22


In [34]:
menu_uri = f's3://{bucket}/{prefix}/menu_codes.csv'
menu_data = pd.read_csv(menu_uri)
menu_data

,Menu_Code,Menu,Price
0,2,Baguette,0.90
1,83,Pain Au Chocolat,1.20
2,82,Pain,1.15
3,138,Traditional Baguette,1.20
4,30,Croissant,1.10
...,...,...,...
142,115,Sable F P,1.60
143,91,Pain Suisse Pepito,1.60
144,144,Tulipe,3.50
145,135,Tartelette Cocktail,3.00


In [35]:
test_data_with_predictions=pd.merge(menu_data, test_data_with_predictions, on='Menu_Code', how='outer')
print(test_data_with_predictions)

     Menu_Code        Menu  Price    Year     Month  Day  Avg_temp  \
0            0  12 Macaron   10.0  2022.0  202210.0  1.0      63.4   
1            0  12 Macaron   10.0  2022.0  202210.0  2.0      64.2   
2            0  12 Macaron   10.0  2022.0  202210.0  3.0      60.5   
3            0  12 Macaron   10.0  2022.0  202210.0  4.0      59.4   
4            0  12 Macaron   10.0  2022.0  202210.0  5.0      61.2   
..         ...         ...    ...     ...       ...  ...       ...   
868        146   Vik Bread    2.5  2022.0  202210.0  3.0      60.5   
869        146   Vik Bread    2.5  2022.0  202210.0  4.0      59.4   
870        146   Vik Bread    2.5  2022.0  202210.0  5.0      61.2   
871        146   Vik Bread    2.5  2022.0  202210.0  6.0      56.7   
872        146   Vik Bread    2.5  2022.0  202210.0  7.0      55.1   

     Prediction      Date  
0           1.0  01/10/22  
1           1.0  02/10/22  
2           5.0  03/10/22  
3           9.0  04/10/22  
4           6.0  05

In [36]:
data_with_prediction = test_data_with_predictions[['Date', 'Menu', 'Price', 'Avg_temp', 'Prediction']]

In [38]:
data_with_prediction.to_csv('data_with_prediction.csv', index=False)

In [44]:
#import boto3
s3 = boto3.client('s3')

menu_codes_key = 'data/New_pred2.csv'
bucket_name = 'bakery-project'

csv_data = data_with_prediction.to_csv(index=False, header=False)

# Subir los archivos CSV a S3
s3.upload_file(csv_data, bucket_name, menu_codes_key)

OSError: [Errno 36] File name too long: '01/10/22,12 Macaron,10.0,63.4,1.0\n02/10/22,12 Macaron,10.0,64.2,1.0\n03/10/22,12 Macaron,10.0,60.5,5.0\n04/10/22,12 Macaron,10.0,59.4,9.0\n05/10/22,12 Macaron,10.0,61.2,6.0\n06/10/22,12 Macaron,10.0,56.7,7.0\n07/10/22,12 Macaron,10.0,55.1,6.0\n,Armoricain,2.5,,\n01/10/22,Baguette,0.9,63.4,43.0\n02/10/22,Baguette,0.9,64.2,35.0\n03/10/22,Baguette,0.9,60.5,27.0\n04/10/22,Baguette,0.9,59.4,29.0\n05/10/22,Baguette,0.9,61.2,22.0\n06/10/22,Baguette,0.9,56.7,22.0\n07/10/22,Baguette,0.9,55.1,23.0\n01/10/22,Baguette Apero,4.5,63.4,0.0\n02/10/22,Baguette Apero,4.5,64.2,-1.0\n03/10/22,Baguette Apero,4.5,60.5,2.0\n04/10/22,Baguette Apero,4.5,59.4,3.0\n05/10/22,Baguette Apero,4.5,61.2,2.0\n06/10/22,Baguette Apero,4.5,56.7,2.0\n07/10/22,Baguette Apero,4.5,55.1,1.0\n01/10/22,Baguette Graine,1.3,63.4,2.0\n02/10/22,Baguette Graine,1.3,64.2,2.0\n03/10/22,Baguette Graine,1.3,60.5,4.0\n04/10/22,Baguette Graine,1.3,59.4,6.0\n05/10/22,Baguette Graine,1.3,61.2,3.0\n06

In [ ]:
predictor.delete_endpoint()